# Cytosim reporting for multiple parameter runs

In [7]:
## By Jennifer Hill, 2021
## Adapted from Akamatsu et al., 2020, eLife
## Report solid, fiber, crosslinker, and arp2/3 positions, numbers, and states from cytosim runs

#import packages and libraries
import numpy as np
from scipy.stats import kde
from scipy.ndimage import gaussian_filter
import pandas as pd
import os
import shutil
import subprocess
from subprocess import Popen
import datetime
import matplotlib.pyplot as plt  # plotting
plt.style.use('seaborn-v0_8-colorblind') # set plot style
plt.cool()                          # heatmap color scheme
%matplotlib inline

<Figure size 640x480 with 0 Axes>

In [8]:
#set variables to define what will be reported
timestep = 0.0001
report_solid = 'yes'
report_fibers = 'no'
report_xlinks = 'no'
report_arp = 'no'

save_pickles ='yes'

In [9]:
#set output directory where results will be placed (same directory that contains run directories) and working directory
output_dir = '/Users/jenniferhill/Documents/tubeZsavio/experiments/longbudtime_1400pN_output_12031735/'
working_dir = '/Users/jenniferhill/Documents/tubeZsavio/'


#set location of report executable
report_loc = '/Users/jenniferhill/Documents/tubeZlemmy/bin/report'

os.chdir(working_dir)

In [10]:
# make prefix for figure filenames
now = datetime.datetime.now()
date = now.strftime('%Y%m%d')
pref = date

In [11]:
# make a dataframes and figures folder, in the output folder specific to this set of simulations. will take a lot of space for lots of sets of simulations.
    
os.chdir(output_dir)

if os.path.isdir('figures') == False:
    os.mkdir('figures')

if os.path.isdir('dataframes') == False:
    os.mkdir('dataframes')
    
os.chdir(working_dir)

In [12]:
#define dictionaries for datasets and a list for run directories
solid_allruns = dict() #bud positions

fiber_forces_allruns = dict() #forces on actin filaments
fiber_ends_allruns = dict() #positions of plus/minus ends

xlinks_allruns = dict() #positions of xlinks and arp2/3s
xlinks_state_allruns = dict() #binding state of xlinks
xlinks_forces_allruns = dict() #forces on xlinks

arp_allruns = dict() #positions of xlinks and arp2/3s
arp_branches_allruns = dict() #branching angle of arp2/3s

properties_allruns = dict()
rundirs = []

In [13]:
for rundir in os.listdir(output_dir):
# for rundir in ['xlinks_output_7150999']:
     if rundir.startswith('run0'):
        os.chdir(output_dir+'/'+rundir)
        #change directory to ['xlinks_output_7150999/run****_****']
        if report_solid == 'yes':
            subprocess.call([report_loc, 'solid', 'output=solid.txt']) #runs the command report_loc (bin/report) with args 'solid' and 'output=solid_jl.txt'
            
            solid = open('solid.txt', 'r')
            solid_allruns[rundir] = solid.readlines() #read the lines of solid.txt into a list called run****_**** and add to the solid_allruns dictionary
            solid.close()  
        
        if report_fibers == 'yes':
            subprocess.call([report_loc, 'fiber:forces', 'output=fiber_forces.txt'])
            subprocess.call([report_loc, 'fiber:ends', 'output=fiber_ends.txt'])
            
            fiber_forces = open('fiber_forces.txt', 'r')
            fiber_forces_allruns[rundir] = fiber_forces.readlines() #read the lines of fiber_forces.txt into a list called run****_**** and add to the fiber_forces_allruns dictionary
            fiber_forces.close()
            
            fiber_ends = open('fiber_ends.txt', 'r')
            fiber_ends_allruns[rundir] = fiber_ends.readlines() #read the lines of fiber_ends.txt into a list called run****_**** and add to the fiber_ends_allruns dictionary
            fiber_ends.close()   
            
        if report_xlinks == 'yes':
            subprocess.call([report_loc, 'couple:state', 'output=couple.txt'])
            subprocess.call([report_loc, 'couple', 'output=couple_state.txt']) 
            subprocess.call([report_loc, 'couple:bridge', 'output=couple_forces.txt'])

            xlinks = open('couple.txt', 'r')
            xlinks_allruns[rundir] = xlinks.readlines()
            xlinks.close()
            
            xlinks_state = open('couple_state.txt', 'r')
            xlinks_state_allruns[rundir] = xlinks_state.readlines()
            xlinks_state.close()
        
            xlinks_forces = open('couple_forces.txt', 'r')
            xlinks_forces_allruns[rundir] = xlinks_forces.readlines()
            xlinks_forces.close()
        
        if report_arp == 'yes':
            subprocess.call([report_loc, 'couple:state', 'output=couple_state.txt']) 
            subprocess.call([report_loc, 'couple:angle:arp23', 'output=couple_arp_angles.txt'])
    
            arp = open('couple_state.txt', 'r')
            arp_allruns[rundir] = arp.readlines()
            arp.close()
            
            arp_branch = open('couple_arp_angles.txt', 'r')
            arp_branches_allruns[rundir] = arp_branch.readlines()
            arp_branch.close()
        
        properties = open('properties.cmo', 'r')
        properties_allruns[rundir] = properties.readlines() #read the lines of properties.cmo into a list called run****_**** and add to the properties_allruns dictionary
        properties.close()
        rundirs.append(rundir) #add the current rundir run****_**** to the list of run directories
        print('finished reporting ' + rundir)
        os.chdir(working_dir)

In [14]:
properties_dict_allruns = dict()

for rundir in rundirs:
    properties = properties_allruns[rundir] #make a list called properties that is the lines from properties.cmo for that run
    properties_dict = dict()
    for line in properties:
        if '=' in line:
            line = line.strip().split(' = ') 
            #get rid of spaces on either end of the line, convert line into a list with items separated by an = in the line
            #ex: ' time       = 9;' strip -> 'time     = 9;' split(' = ') -> ['time    ','9;']
            properties_dict[line[0].strip()] = line[-1].strip(';')
            #for the item (parameter) in the properties dictionary, add whatever the value is for that run
            #ex: line[0].strip() is 'time', the value for time in the properties dictionary is '9'
        properties_dict_allruns[rundir] = properties_dict #append the properties dictionary for this run to the dictionary with properties for all runs
    print('finished reading ' + rundir + ' properties')   
        
properties_df = pd.DataFrame.from_dict(properties_dict_allruns, orient = 'index')     

## Parse bud positions

In [15]:
#parse bud position
all_solid_outputs_allruns = pd.DataFrame()

if report_solid == 'yes':
    solid_outputs_allruns = []
    for rundir in rundirs:
        all_lines = solid_allruns[rundir] #all_lines is the list of lines read from solid.txt for this run
        timepoints = []
        outputs = []
        for line in all_lines:
            line = line.strip() #remove spaces on either end of each line in the list
            if line.startswith('%'):
                if line.startswith('% time'): #happens first for each timepoint
                    time = float(line.split(' ')[-1]) 
                    timepoints.append(time) #for lines that start with % time, split the line into a list and append the time item (-1) as a float to the timepoints list
                    solids = {} #resets solids to empty
                elif line.startswith('% end'): #happens last for each timepoint
                    df = pd.DataFrame.from_dict(solids, orient = 'index') #for lines that start with % end, make a dataframe from the values in solids (points x y and z)
                    outputs.append(df) #append the dataframe of values to the outputs list
            elif len(line.split()) > 0: #aka it contains actual data not empty space
                [solid_class, solid_id, centroid_x, centroid_y, centroid_z,
                point_x, point_y, point_z, idk1, idk2, idk3] = line.split() #split the line into a list of values and assign the values to the associated variables
                solids[int(solid_id)] = {'xpos': float(point_x), 'ypos' : float(point_y),
                                      'zpos' : float(point_z)}
        #creates outputs: list of dataframes containing x, y, and z values for the solid at all time points (one dataframe for each time point)
        all_outputs = pd.concat(outputs, keys = timepoints,
                                names = ['time', 'id'])
        #concatenates all dataframes in the list 'outputs' into a single dataframe with timepoints (one dataframe containing all timepoints)
        solid_outputs_allruns.append(all_outputs)
        
        print( 'finished parsing ' + rundir)

    all_solid_outputs_allruns = pd.concat(solid_outputs_allruns, keys = rundirs,
                                  names = ['run', 'time', 'id'])
all_solid_outputs_allruns.head()

ValueError: No objects to concatenate

In [ ]:
#recalibrate bud z position so it starts at z=0 and is given in nm
if report_solid == 'yes':
    all_solid_outputs_allruns['internalization'] = (all_solid_outputs_allruns['zpos'] + 0.14) * 1000
    all_solid_outputs_allruns.head()

In [ ]:
if report_solid == 'yes':
    if save_pickles=='yes':
        all_solid_outputs_allruns.to_pickle(output_dir+'/dataframes/bud_positions.pkl')

## Parse forces on all fibers

In [ ]:
#parse the forces on all fibers
all_fiber_forces = pd.DataFrame()

if report_fibers == 'yes':
    fiber_forces_outputs_allruns = [] #make a dictionary for runs where each run is a dataframe of fiber force outputs from that run
    all_fiber_forces = []

    for rundir in rundirs:
        single_all_lines = fiber_forces_allruns[rundir] #define the item in the fiber_forces_allruns dictionary for run****_****
        timepoints = []
        outputs = []
        for line in single_all_lines:
            line = line.strip()
            if line.startswith('%'):
                if line.startswith('% start'): #add the timepoint to the timepoints list, re-empty singles
                    time = float(line.split(' ')[-1])
                    timepoints.append(time)
                    singles = {}
                elif line.startswith('% end'): #make a dataframe from the dictionary of force values, append it to the outputs list
                    df = pd.DataFrame.from_dict(singles, orient = 'index')
                    outputs.append(df)
                    #print('finished parsing ' + rundir + ' timepoint ' + str(time))
            elif len(line.split()) > 0: #for lines with data in them, split the data into individual values
                [fiber_id, pt_index, xpos, ypos, zpos,
                xforce, yforce, zforce, tension] = line.split()
                singles[str(fiber_id)+'_'+str(pt_index)] = {'fiber_id' : int(fiber_id),'pt_index' : int(pt_index),
                                                            'xpos': float(xpos), 'ypos' : float(ypos), 'zpos': float(zpos),
                                                            'xforce' : float(xforce), 'yforce' : float(yforce),
                                                            'zforce': float(zforce), 'tension': float(tension)}

        #concatenate all outputs dataframes (values for a given timepoint) into a single dataframe with values for all timepoints
        all_outputs = pd.concat(outputs, keys = timepoints,
                                names = ['time', 'id'])
        #magnitude of the force is sqrt of dir. vectors squared
        all_outputs['force_magnitude'] = np.sqrt(np.square(all_outputs['xforce']) + 
                                                  np.square(all_outputs['yforce']) +
                                                  np.square(all_outputs['zforce']))
        #add the outputs dataframe for the run to the list of dataframes for all runs
        fiber_forces_outputs_allruns.append(all_outputs)

        print( 'finished parsing ' + rundir)

    #concatenate dataframes from each run into a single dataframe with values for all timepoints for all runs
    all_fiber_forces = pd.concat(fiber_forces_outputs_allruns, keys = rundirs,
                                      names = ['run', 'time', 'id'])
all_fiber_forces.head()


In [ ]:
if report_fibers == 'yes':
    if save_pickles=='yes':
        all_fiber_forces.to_pickle(output_dir+'/dataframes/actin_positions_forces.pkl')

## Parse fiber end positions

In [ ]:
#parse fiber end positions
all_fiber_ends = pd.DataFrame()

if report_fibers == 'yes':
    fiber_ends_outputs_allruns = []
    for rundir in rundirs:
        single_all_lines = fiber_ends_allruns[rundir] #define the item in the fiber_ends_allruns dictionary for run****_****
        timepoints = []
        outputs = []
        for line in single_all_lines:
            line = line.strip()
            if line.startswith('%'): 
                if line.startswith('% time'): #add the timepoint to the timepoints list, re-empty singles
                    time = float(line.split(' ')[-1])
                    timepoints.append(time)
                    singles = {}
                elif line.startswith('% end'): #make a dataframe from the dictionary of endpoint values, append it to the outputs list
                    df = pd.DataFrame.from_dict(singles, orient = 'index')
                    outputs.append(df)
                    # print 'finished parsing ' + rundir + ' timepoint ' + str(time)
            elif len(line.split()) > 0: #for lines with data in them, split the data into individual values
                [fiber_class, fiber_id, length, minus_state, minus_xpos, minus_ypos, minus_zpos,
                minus_xdir, minus_ydir, minus_zdir, plus_state, plus_xpos, plus_ypos,
                plus_zpos, plus_xdir, plus_ydir, plus_zdir] = line.split()
                singles[int(fiber_id)] = {'fiber_id' : int(fiber_id), 'length':float(length),
                                          'minus_state':int(minus_state), 'minus_xpos':float(minus_xpos),
                                          'minus_ypos':float(minus_ypos), 'minus_zpos':float(minus_zpos),
                                          'minus_xdir':float(minus_xdir), 'minus_ydir':float(minus_ydir),
                                          'minus_zdir':float(minus_zdir), 'plus_state':int(plus_state),
                                          'plus_xpos':float(plus_xpos), 'plus_ypos':float(plus_ypos),
                                          'plus_zpos':float(plus_zpos), 'plus_xdir':float(plus_xdir),
                                          'plus_ydir':float(plus_ydir), 'plus_zdir':float(plus_zdir)}

        #concatenate all outputs dataframes (values for a given timepoint) into a single dataframe with values for all timepoints
        all_outputs = pd.concat(outputs, keys = timepoints,
                            names = ['time', 'id'])
        #convert plus end position in cartesian coords to polar coords (just r)
        all_outputs['plus_rpos'] = np.sqrt(np.square(all_outputs['plus_xpos']) +
                                      np.square(all_outputs['plus_ypos']))

        #convert zdir to degrees, oriented vertically such that +90 is POSITIVE orientation and -90 is negative orientation
        all_outputs['zdir_deg_flip90'] = np.degrees((np.arccos(all_outputs['plus_zdir'])-(np.pi)/2))

        #add the outputs dataframe for the run to the list of dataframes for all runs
        fiber_ends_outputs_allruns.append(all_outputs)

        print( 'finished parsing ' + rundir)

    #concatenate dataframes from each run into a single dataframe with values for all timepoints for all runs
    all_fiber_ends = pd.concat(fiber_ends_outputs_allruns, keys = rundirs,
                                  names = ['run', 'time', 'id'])
all_fiber_ends.head()

## Recalibrate fiber end positions
### Center X, Y, Z, R positions around bud position

In [ ]:
#bud moves in x, y, and z during simulation, recalculate fiber x, y, and z positions to be relative to bud position
all_fiber_ends_recal = pd.DataFrame()

if report_fibers == 'yes' and report_solid == 'yes':
    all_fiber_ends_recal = pd.merge(all_fiber_ends.reset_index(), all_solid_outputs_allruns.reset_index(), on = ['run','time'])

    all_fiber_ends_recal['minus_xpos_recal'] = (all_fiber_ends_recal['minus_xpos'] - all_fiber_ends_recal['xpos'])*1000
    all_fiber_ends_recal['minus_ypos_recal'] = (all_fiber_ends_recal['minus_ypos'] - all_fiber_ends_recal['ypos'])*1000
    all_fiber_ends_recal['minus_zpos_recal'] = (all_fiber_ends_recal['minus_zpos'] + 0.03)*(-1000)

    all_fiber_ends_recal['plus_xpos_recal'] = (all_fiber_ends_recal['plus_xpos'] - all_fiber_ends_recal['xpos'])*1000
    all_fiber_ends_recal['plus_ypos_recal'] = (all_fiber_ends_recal['plus_ypos'] - all_fiber_ends_recal['ypos'])*1000
    all_fiber_ends_recal['plus_zpos_recal'] = (all_fiber_ends_recal['plus_zpos'] +0.03)*(-1000)

    all_fiber_ends_recal['plus_rpos_recal'] = np.sqrt(np.square(all_fiber_ends_recal['plus_xpos_recal']) +
                                      np.square(all_fiber_ends_recal['plus_ypos_recal']))

    all_fiber_ends_recal['minus_rpos_recal'] = np.sqrt(np.square(all_fiber_ends_recal['minus_xpos_recal']) +
                                      np.square(all_fiber_ends_recal['minus_ypos_recal']))

    all_fiber_ends_recal = all_fiber_ends_recal.drop(columns=['id_y', 'xpos', 'ypos', 'zpos', 'internalization'])
    all_fiber_ends_recal = all_fiber_ends_recal.rename(index=str, columns={"id_x": "id"})
    all_fiber_ends_recal = all_fiber_ends_recal.set_index(['run', 'time', 'id'])

all_fiber_ends_recal.head()

In [ ]:
if save_pickles=='yes':
    if report_fibers == 'yes' and report_solid == 'yes':
        all_fiber_ends_recal.to_pickle(output_dir+'/dataframes/actin_plus_minus_ends_recal.pkl')
    elif report_fibers == 'yes':
        all_fiber_ends.to_pickle(output_dir+'/dataframes/actin_plus_minus_ends.pkl')

In [ ]:
#bud_positions, internalization: true z position in nm of tip of bud, starts at z=0 and increases
#all_fiber_ends_recal, x,y,z,rpos_recal: true x,y,z,r position in nm of plus/minus ends of actin filaments

## Report and parse crosslinkers

In [ ]:
#parse crosslinker positions
all_couple_xlinks = pd.DataFrame()

if report_xlinks == 'yes':
    couple_xlinks_outputs_allruns = [] #make a dictionary for runs where each run is a dataframe of xlinks outputs from that run
    for rundir in rundirs:
        couple_all_lines = xlinks_allruns[rundir] #define couple_all_lines as the list of lines for run****_**** from xlinks_allruns dictionary
        timepoints = []
        outputs = []
        for line in couple_all_lines:
            line = line.strip()
            if line.startswith('%'): #see same lines from parsing internalization
                if line.startswith('% time'):
                    time = float(line.split(' ')[-1])
                    timepoints.append(time)
                    couples_xlinks = {} #resets couples_xlinks to empty
                elif line.startswith('% end'):
                    df = pd.DataFrame.from_dict(couples_xlinks, orient = 'index') #creates a dataframe consisting of all the lines from the last time point
                    outputs.append(df) #adds the dataframe to the outputs list
                    # print 'finished parsing ' + rundir + ' timepoint ' + str(time)
            elif line.startswith('2'): #line is for a couple of class 2, aka crosslinker
                [couple_class, couple_id, bound_state, xpos, ypos, zpos, id_fiber1, abscissa1, id_fiber2, abscissa2] = line.split()
                couples_xlinks[int(couple_id)] = {'bound_state' : int(bound_state), 'arp_id': int(couple_id), 
                                               'xpos': float(xpos), 'ypos': float(ypos), 'zpos': float(zpos), 
                                               'id_fiber1': int(id_fiber1), 'abscissa1': float(abscissa1), 'id_fiber2': int(id_fiber2), 'abscissa2': float(abscissa2)}
                #for the list item in couples_xlinks defined by the couple_id, assign the values to the given names

        #convert the list of outputs dataframes to a single dataframe with xlinks outputs for each timepoint
        all_outputs = pd.concat(outputs, keys = timepoints,
                            names = ['time', 'id'])

        #add the dataframe with outputs for all time points for run****_**** to the list of outputs for each run
        couple_xlinks_outputs_allruns.append(all_outputs)

        print( 'finished parsing ' + rundir)

    #convert the list of outputs for each run into a single dataframe containing all runs
    all_couple_xlinks = pd.concat(couple_xlinks_outputs_allruns, keys = rundirs,
                                  names = ['run', 'time', 'id'])

all_couple_xlinks.head()

In [ ]:
#create a second dataframe for singly bound crosslinkers only
singly_bound_xlinks = pd.DataFrame()

if report_xlinks == 'yes':
    singly_bound = []

    #id_fiber is zero when xlink hand is not bound to a fiber, singly bound will have one id = 0 and the other id =/= 0
    singly_bound.append(all_couple_xlinks.loc[np.logical_and(all_couple_xlinks['id_fiber1'] != 0, all_couple_xlinks['id_fiber2'] == 0)])
    #append the dataframe made up of the rows from all_couple_xlinks where id_fiber1 =/= 0 and id_fiber2 = 0 to the list singly_bound
    singly_bound.append(all_couple_xlinks.loc[np.logical_and(all_couple_xlinks['id_fiber1'] == 0, all_couple_xlinks['id_fiber2'] != 0)])
    #append the dataframe made up of the rows from all_couple_xlinks where id_fiber1 = 0 and id_fiber2 =/= 0 to the list singly_bound

    #concatenate the two dataframes in the list together and sort by the run index (sorts all other indices as well)
    singly_bound_xlinks = pd.concat(singly_bound)
    singly_bound_xlinks = singly_bound_xlinks.sort_index()
singly_bound_xlinks.head()

In [ ]:
#create a third dataframe for active crosslinkers only
active_xlinks = pd.DataFrame()

if report_xlinks == 'yes':
    active_xlinks = pd.DataFrame()
    active_xlinks = all_couple_xlinks.loc[np.logical_and(all_couple_xlinks['id_fiber1'] != 0, all_couple_xlinks['id_fiber2'] != 0)]
    #doubly bound will have both fiber_ids =/= 0
active_xlinks.head()

In [ ]:
#save to pickles
if save_pickles=='yes':
    singly_bound_xlinks.to_pickle(output_dir+'/dataframes/singly_bound_xlinks.pkl')
    active_xlinks.to_pickle(output_dir+'/dataframes/active_xlinks.pkl')

In [ ]:
#parse crosslinker states
all_couple_xlinks_state = pd.DataFrame()

if report_xlinks == 'yes':
    xlinks_state_outputs_allruns = [] #make a dictionary for runs where each run is a dataframe of xlinks outputs from that run
    for rundir in rundirs:
        couple_all_lines = xlinks_state_allruns[rundir] #define couple_all_lines as the list of lines for run****_**** from xlinks_state_allruns dictionary
        timepoints = []
        outputs = []
        for line in couple_all_lines:
            line = line.strip()
            if line.startswith('%'): #see same lines from parsing internalization
                if line.startswith('% time'):
                    time = float(line.split(' ')[-1])
                    timepoints.append(time)
                    couples_xlinks = {} #resets couples_xlinks dict to empty
                elif line.startswith('% end'):
                    df = pd.DataFrame.from_dict(couples_xlinks, orient = 'index') #creates a dataframe consisting of crosslinker line from the last time point
                    outputs.append(df) #adds the dataframe to the outputs list
                    # print 'finished parsing ' + rundir + ' timepoint ' + str(time)
            elif line.startswith('crosslinker'): #line is for a crosslinkers not arp2/3
                [couple, total, active, FF, AF, FA, AA] = line.split()
                couples_xlinks[(0)] = {'couple' : str(couple), 'total': int(total), 
                                               'active': int(active), 'FF': int(FF), 'AF': int(AF), 
                                               'FA': int(FA), 'AA': int(AA)}
                #for the dict item in couples_xlinks defined by the couple_id, assign the values to the given names

        #convert the list of outputs dataframes to a single dataframe with xlinks outputs for each timepoint
        all_outputs = pd.concat(outputs, keys = timepoints,
                            names = ['time', 'id'])

        #generate a bound column with the total number of crosslinkers bound in the network
        all_outputs['bound'] = all_outputs['AF'] + all_outputs['FA'] + all_outputs['AA']

        #add the dataframe with outputs for all time points for run****_**** to the list of outputs for each run
        xlinks_state_outputs_allruns.append(all_outputs)

        print( 'finished parsing ' + rundir)

    #convert the list of outputs for each run into a single dataframe containing all runs
    all_couple_xlinks_state = pd.concat(xlinks_state_outputs_allruns, keys = rundirs,
                                  names = ['run', 'time', 'id'])

all_couple_xlinks_state.head()


In [ ]:
#parse crosslinker forces
all_couple_xlinks_force = pd.DataFrame()

if report_xlinks == 'yes':
    xlinks_force_outputs_allruns = [] #make a dictionary for runs where each run is a dataframe of xlinks outputs from that run
    for rundir in rundirs:
        couple_all_lines = xlinks_forces_allruns[rundir] #define couple_all_lines as the list of lines for run****_**** from xlinks_forces_allruns dictionary
        timepoints = []
        outputs = []
        for line in couple_all_lines:
            line = line.strip()
            if line.startswith('%'): #see same lines from parsing internalization
                if line.startswith('% time'):
                    time = float(line.split(' ')[-1])
                    timepoints.append(time)
                    couples_xlinks = {} #resets couples_xlinks dict to empty
                elif line.startswith('% end'):
                    df = pd.DataFrame.from_dict(couples_xlinks, orient = 'index') #creates a dataframe consisting of crosslinker line from the last time point
                    outputs.append(df) #adds the dataframe to the outputs list
                    # print 'finished parsing ' + rundir + ' timepoint ' + str(time)
            elif line.startswith('2'): #line is for a couple of class crosslinker
                [couple_class, couple_id, id_fiber1, abscissa1, id_fiber_2, abscissa2, force_nrm, 
                 xpos_hand1, ypos_hand1, zpos_hand1, xpos_hand2, ypos_hand2, zpos_hand2] = line.split()
                couples_xlinks[int(couple_id)] = {'couple_class' : int(couple_class), 'couple_id': int(couple_id), 
                                               'id_fiber1': int(id_fiber1), 'abscissa1': float(abscissa1), 'id_fiber2': int(id_fiber2), 'abscissa2': float(abscissa2), 
                                               'force_nrm': float(force_nrm), 'xpos_hand1': float(xpos_hand1), 'ypos_hand1': float(ypos_hand1), 'zpos_hand1': float(zpos_hand1),
                                                 'xpos_hand2': float(xpos_hand2), 'ypos_hand2': float(ypos_hand2), 'zpos_hand2': float(zpos_hand2),}
                #for the dict item in couples_xlinks defined by the couple_id, assign the values to the given names

        #convert the list of outputs dataframes to a single dataframe with xlinks outputs for each timepoint
        all_outputs = pd.concat(outputs, keys = timepoints,
                            names = ['time', 'id'])

        #add the dataframe with outputs for all time points for run****_**** to the list of outputs for each run
        xlinks_force_outputs_allruns.append(all_outputs)

        print( 'finished parsing ' + rundir)

    #convert the list of outputs for each run into a single dataframe containing all runs
    all_couple_xlinks_force = pd.concat(xlinks_force_outputs_allruns, keys = rundirs,
                                  names = ['run', 'time', 'id'])

all_couple_xlinks_force.head()

In [ ]:
#merge xlinks forces to active_xlinks
if report_xlinks == 'yes':
    force_nrm = all_couple_xlinks_force.loc[:,('force_nrm')]
    active_xlinks["force_nrm"] = force_nrm
active_xlinks.head()

In [ ]:
#save to pickles
if report_xlinks == 'yes':
    if save_pickles=='yes':
        all_couple_xlinks_state.to_pickle(output_dir+'/dataframes/xlinks_state.pkl')
        all_couple_xlinks.to_pickle(output_dir+'/dataframes/all_xlinks.pkl')

## Report and parse Arp2/3

In [ ]:
#parse the positions of all and bound arp2/3s
all_couple_arp = pd.DataFrame()

if report_arp == 'yes':
    couple_arp_outputs_allruns = [] #make a dictionary for runs where each run is a dataframe of arp2/3 outputs from that run
    for rundir in rundirs:
        couple_all_lines = arp_allruns[rundir] #define couple_all_runs as the list of lines for run****_**** from arp_allruns dictionary
        timepoints = []
        outputs = []
        for line in couple_all_lines:
            line = line.strip()
            if line.startswith('%'): #see same lines from parsing internalization
                if line.startswith('% time'):
                    time = float(line.split(' ')[-1])
                    timepoints.append(time)
                    couples_arp = {} #resets couples_arp to empty
                elif line.startswith('% end'):
                    df = pd.DataFrame.from_dict(couples_arp, orient = 'index') #creates a dataframe consisting of all the lines from the last time point
                    outputs.append(df) #adds the dataframe to the outputs list
                    # print 'finished parsing ' + rundir + ' timepoint ' + str(time)
            elif line.startswith('1'): #line is for a couple of class 1, aka arp2/3
                [couple_class, couple_id, bound_state, xpos, ypos, zpos, id_fiber1, abscissa1, id_fiber2, abscissa2] = line.split()
                couples_arp[int(couple_id)] = {'bound_state' : int(bound_state), 'arp_id': int(couple_id), 
                                               'xpos': float(xpos), 'ypos': float(ypos), 'zpos': float(zpos), 
                                               'id_fiber1': int(id_fiber1), 'abscissa1': float(abscissa1), 'id_fiber2': int(id_fiber2), 'abscissa2': float(abscissa2)}
                #for the list item in couples_arp defined by the couple_id, assign the values to the given names

        #convert the list of outputs dataframes to a single dataframe with arp2/3 outputs for each timepoint
        all_outputs = pd.concat(outputs, keys = timepoints,
                            names = ['time', 'id'])

        #add the dataframe with outputs for all time points for run****_**** to the list of outputs for each run
        couple_arp_outputs_allruns.append(all_outputs)

        print( 'finished parsing ' + rundir)

    #convert the list of outputs for each run into a single dataframe containing all runs
    all_couple_arp = pd.concat(couple_arp_outputs_allruns, keys = rundirs,
                                  names = ['run', 'time', 'id'])

    #create a second dataframe for bound arp2/3 only
    bound_arp = all_couple_arp.loc[all_couple_arp['id_fiber1'] != 0] #id_fiber1 is zero when arp2/3 is not bound to a fiber
all_couple_arp.head() #show the first few lines fo the all_couple_arp dataframe 

In [ ]:
#parse arp2/3 branch angles (angle in cos theta)
all_arp_combined = pd.DataFrame()

if report_arp == 'yes':
    couple_arp_branches_outputs_allruns = []

    for rundir in rundirs:
        couple_branches_all_lines = arp_branches_allruns[rundir] 
        #couple_branches_all_lines is the item in the arp_branches_allruns dictionary for run****_****
        timepoints = []
        outputs = []

        for line in couple_branches_all_lines:
            line = line.strip()
            if line.startswith('%'):
                if line.startswith('% time'): #add the timepoint to the timepoints list, re-empty couples_arp_branch
                    time = float(line.split(' ')[-1])
                    timepoints.append(time)
                    couples_arp_branch = {}
                elif line.startswith('% end'): #make a dataframe from the dictionary of scal_prod values, append it to the outputs list
                    df = pd.DataFrame.from_dict(couples_arp_branch, orient = 'index')
                    outputs.append(df)
            elif len(line.split()) > 0: #for lines with data in them, split the data into class, id, and scal_prod
                [couple_class, couple_id, scal_prod] = line.split()
                couples_arp_branch[int(couple_id)] = {'scal_prod' : float(scal_prod)}
                #for the item in the dictionary called couple_id, add the scal_prod value

        #concatenate all outputs dataframes (scal_prod values for a given timepoint) into a single dataframe with scal_prod values for all timepoints
        all_outputs = pd.concat(outputs, keys = timepoints,
                            names = ['time', 'id'])

        #convert scal_prod values into a branch angle in degrees (scal_prod is dot product using unit vectors, so angle is cos-1(scal_prod)
        all_outputs['branch_angle_deg'] = np.degrees(np.arccos(all_outputs['scal_prod']))
        #add the outputs dataframe for the run to the list of dataframes for all runs
        couple_arp_branches_outputs_allruns.append(all_outputs)
        print( 'finished parsing ' + rundir)

    #concatenate dataframes from each run into a single dataframe with values for all timepoints for all runs
    all_couple_arp_branches = pd.concat(couple_arp_branches_outputs_allruns, keys = rundirs,
                                  names = ['run', 'time', 'id'])

    #merge the branching angle dataframe with the arp23 dataframe
    #how=outer means that for the arps without information about branching, these arps are kept in the dataframe
    all_arp_combined = pd.merge(all_couple_arp, all_couple_arp_branches, on = ['run','time', 'id'], how = 'outer')

all_arp_combined.head()


In [ ]:
if report_arp == 'yes':
    if save_pickles=='yes':
        all_arp_combined.to_pickle(output_dir+'/dataframes/arp_positions_angles.pkl')
        bound_arp.to_pickle(output_dir+'/dataframes/bound_arp.pkl')

In [ ]:
print('All done!')
print('Plot using CME_plotting_single_rungroup.ipynb, CME_plotting_multi_rungroup.ipynb, or CME_summary_plotting.ipynb')